In [1]:
clean_up = True
%run StdPackages.ipynb
repo = os.path.join(d['main'],'gams\\AbatementExample')
db_str = os.path.join(repo,'V01_DB.gdx')

The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb14.gdx is still active and was not deleted.
The file_gams_py_gdb15.gdx is still active and was not deleted.
The file_gams_py_gdb16.gdx is still active and was not deleted.
The file_gams_py_gdb17.gdx is still active and was not deleted.
The file_gams_py_gdb18.gdx is still active and was not deleted.
The file_gams_py_gdb19.gdx is still active and was not deleted.
The file_gams_py_gdb20.gdx is still active and was not deleted.
The file_gams_py_gdb21.gdx is still active and was not deleted.
The file_gams_py_gdb22.gdx is still active and was not deleted.
The file_gams_py_gdb23.gdx is still active and was not deleted.
The file_gams_py_gdb24.gdx is still active and was not deleted.
The file_gams_py_gdb25.gdx is still active and was not deleted.
The file_gams_py_gdb26.gdx is still active and was not deleted.
The file_gams_py_gdb27.gdx is still active and was not deleted.


# *GmsPy.GmsSettings*

*A class of settings used to specify gams models.*

The class is initialized either with kwargs or from a pickle. If it is not initialized from a pickle, a number of default attributes are defined.

```python
    class GmsPy.GmsSettings:
        def __init__(self, file_path = None, **kwargs):
```
If a file path is not supplied, the kwargs are merged with default settings. Default settings:
* ```self.name:``` Name of the settings file (a string).
* ```self.macros:``` Empty dictionary. A container to capture the declared macros (GAMS macros).
* ```self.Precompiler:``` An instance of the Precompiler class from the ```dreamtools``` package (gamY).
* ```self.Compile:``` A user-class defined to write gams code from group/model specifications. 
* ```self.db:``` A ```GpyDB``` database instance. 
* ```self.from_db:``` Specifying how much of Gams code should be written automatically from the database in ```self.db```.
* ```self.locals:``` Dictionary with local variables to be added as placeholders when running the gams model.
* ```self.states:``` Dictionary with different model states (e.g. different selection of endogenous groups, blocks etc.).
* ```self.state:``` The current state of the model (key in self.states).

## Example 1: Use gamY model

This example shows how to use gamY model blocks to create a model. We show basic methods for reading and writing model components. Specifically we cover:
1. Specify gamY files to be included by the model (various ways of parsing the argument to the gamY Precompiler). Use these arguments to write a gams file. 
2. Use information on endogenous groups, exogeonus groups, and model blocks to create an instance of the model, and use the GmsSettings class to automate running this.

### 1.1. Add gamY files

In [2]:
s = GmsPy.GmsSettings(**{'name': 'Test1'})

For this specific example, we need to specify the technology type used in the model: The gamY model code refers to a technology type using the syntax "%techtype%". We add this to the local namespace:

In [3]:
s.Precompiler.locals = {'techtype': "'logit'"}

One way to use the ```GmsSettings``` class is to read files prepared for the Precompiler in ```dreamtools```. In this example, we read in files that:
1. Define gamY functions and gams macros, 
2. Define and load sets,
3. Define groupings of variables,
4. Define blocks of equations,
5. Load variables as either fixed or unfixed.

We can add files to the ```self.args``` by either (1) passing the file destination or (2) storing the text. Both will eventually be parsed through the gamY precompiler (if no gamY commands are used, this does nothing to the text). 

*The manual way, referring to file path:*

In [4]:
fs = ('V02_functions.gms', 'V02_sets.gms', 'V02_groups.gms', 'V02_blocks.gms', 'V02_groupsload.gms')
files_path = {k: {'file_path': os.path.join(repo,k)} for k in fs}

*The manual way, referring to string:*

In [5]:
def read(x,path = repo):
    with open(os.path.join(path,x), 'r') as f:
        return f.read()
files_string = {k: read(k) for k in fs}

*Either one can be added to the args. Using file names builds in a dependency on the files that persists if we store the ```GmsSettings``` instance for later use:*

In [6]:
s['args'] = files_path

The class uses the method ```self.write``` to write the components in ```self.args``` into a dictionary of strings. Depending on the argument, it might be processed by the Precompiler, or simply copying the text.

In [7]:
%%capture
s.write()

### 1.1.1 Precompiler settings

It is important to note that the ```Precompiler``` does not have separate read/write methods: When it reads a file/text string in order to define groups and model blocks, it automatically writes the corresponding text that declare these: This means that if we apply the ```Precompiler``` to a file, it will produce one piece of text (declaring models/groups), but if we do it again (without resetting the Precompiler settings), it will behave differently, as it has already written definitions for the relevant groups/model blocks once. 

In [8]:
print(s['text']['V02_groups.gms'][0:600])


# Non-Technology groups:

# ----------------------------------------G_V01_NT_endo_always----------------------------------------
#  Initialize G_V01_NT_endo_always group
# ----------------------------------------------------------------------------------------------------
$offlisting
VARIABLE pS[s,n] "cost price index for outputs";
pS.L[s,n]$(V01_NT_out[s,n]) = 0;
VARIABLE pD[s,n] "cost price index for intermediates";
pD.L[s,n]$(V01_NT_int[s,n]) = 0;
VARIABLE qD[s,n] "demand for pure inputs outside Tech";
qD.L[s,n]$(V01_NT_x[s,n]) = 0;
$onlisting


# -----------------------------------------G


In [9]:
text = s.Precompiler(**files_path['V02_groups.gms'])
print(text[0:400])


# Non-Technology groups:

# ----------------------------------------G_V01_NT_endo_always----------------------------------------
#  Initialize G_V01_NT_endo_always group
# ----------------------------------------------------------------------------------------------------
$offlisting
$onlisting


# -----------------------------------------G_V01_NT_endo_base----------------------------------------


Because of this feature of the ```Precompiler```, the ```self.write(reset=True)``` method automatically uses the setting ```reset=True```, which resets the Precompiler before processing the ```self.args```. Thus, re-running the ```self.write``` method produces the same result as the first time:

In [10]:
s.write()
print(s['text']['V02_groups.gms'][0:600])


# Non-Technology groups:

# ----------------------------------------G_V01_NT_endo_always----------------------------------------
#  Initialize G_V01_NT_endo_always group
# ----------------------------------------------------------------------------------------------------
$offlisting
VARIABLE pS[s,n] "cost price index for outputs";
pS.L[s,n]$(V01_NT_out[s,n]) = 0;
VARIABLE pD[s,n] "cost price index for intermediates";
pD.L[s,n]$(V01_NT_int[s,n]) = 0;
VARIABLE qD[s,n] "demand for pure inputs outside Tech";
qD.L[s,n]$(V01_NT_x[s,n]) = 0;
$onlisting


# -----------------------------------------G


### 1.2. Model instances and run files

So far, the gamY files does not include the "run" part of the gams code: This is the part of the code that (1) Fix/unfix relevant variables, (2) Define models from blocks of equations, and (3) Executes the solve statement. We can of course provide these using gamY files as well, or we can use the ```GmsSettings``` to write these arguments for us. To use class to write the run file for us, we need to specify the relevant information:

*Specify endogenous groups, exogeonous groups, blocks:*

In [11]:
s['g_endo'] = ['G_V01_NT_endo_always', 'G_V01_NT_endo_base', 'G_V01_T_endo_always', 'G_V01_T_endo_base', 'G_V01_ACC_endo_base']
s['g_exo'] = ['G_V01_NT_exo_always', 'G_V01_NT_exo_base', 'G_V01_T_exo_always', 'G_V01_T_exo_base']
s['blocks'] = ['M_V01_NT', 'M_V01_T_always', 'M_V01_T_base', 'M_V01_ACC']

*Note that per construction these settings are ordered sets (makes merging easier). Also, the slicing [] syntax uses a simple look-up in the ```self.states[self.state]```, for instance:*

In [12]:
s['g_endo'] is s.states[s.state]['g_endo']

True

*The name of the model in question is per construction the name of the GmsSettings instance + the relevant state. We can, however, also alter this if we want:*

In [13]:
s['name'] = s.name +'_'+s.state # this is the way we define the standard name, so changes nothing here.
s['name']

'Test1_B'

With these specifications, ```GmsWrite``` writes standard components for us, this includes fix, unfix, model, and solve statements:

In [14]:
runargs = GmsWrite.standardRun_gamY(s)
runargs.keys()

dict_keys(['fix', 'unfix', 'model', 'solve'])

*Note: The standard arguments here can be re-run by the Precompiler without resetting and still produce the same results:*

In [15]:
s['args'].update(runargs)
s.write()
s['text'].keys()

dict_keys(['root', 'V02_functions.gms', 'V02_sets.gms', 'V02_groups.gms', 'V02_blocks.gms', 'V02_groupsload.gms', 'fix', 'unfix', 'model', 'solve'])

We can now run the model:

In [16]:
m = GmsPy.GmsModel(ws=d['work'],**{'cns': 'CONOPT4'}) # use CONOPT4 to solve CNS models.
m.addlocal('V01_DB', db_str) # the gams code uses the placeholder %V01_DB% to load data. This directs it to the relevant database.
m.run(run = '\n'.join(s['text'].values()))

## Example 2: Use GpyDB + gamY

In this example, we use the ```GpyDB``` database to write gams text that declares variables. Obviously, if we already have gams code that does this (like in example 1), there is not anything to be gained here. So, we imagine that we have not yet written the files 'V02_sets.gms'. In this case, we need to specify the following inputs: 

In [17]:
db = Database.GpyDB(db=db_str) # database used to declare symbols
functions = read('V02_functions.gms') # read in file that declares functions
declare_groups = read('V02_groups.gms') # read in file that declares groups
blocks = read('V02_blocks.gms') # read in equations for model.

Initialize settings:

In [18]:
s = GmsPy.GmsSettings(**{'name': 'Test2', 'db': db})
s.Precompiler.locals = {'techtype': "'logit'"}

Specify the endogenous/exogenous groups, specify blocks of equations:

In [19]:
s['g_endo'] = ['G_V01_NT_endo_always', 'G_V01_NT_endo_base', 'G_V01_T_endo_always', 'G_V01_T_endo_base', 'G_V01_ACC_endo_base']
s['g_exo'] = ['G_V01_NT_exo_always', 'G_V01_NT_exo_base', 'G_V01_T_exo_always', 'G_V01_T_exo_base']
s['blocks'] = ['M_V01_NT', 'M_V01_T_always', 'M_V01_T_base', 'M_V01_ACC']

We can now collect a standard selection of 'args':

In [20]:
s['args'] = s.stdArgs_gamY(blocks=blocks,functions=functions,declare_groups=declare_groups)
s['args'].keys()

dict_keys(['root', 'Test2_functions', 'Test2_declare', 'Test2_declare_groups', 'Test2_load_groups', 'Test2_blocks', 'Test2_B_fix', 'Test2_B_unfix', 'Test2_B_model', 'Test2_B_solve'])

To solve the model, we simply use the ```self.write``` method again:

In [21]:
%%capture
s.write()

And set up model and solve:

In [22]:
m = GmsPy.GmsModel(ws=d['work'],**{'cns': 'CONOPT4'}) # use CONOPT4 to solve CNS models.
m.addlocal(db.name, db_str)
m.run(run = '\n'.join(s['text'].values()))

## Example 3: Use GpyDB + ```self.Compile```

In this example, we use the class ```_GmsPy.Compile``` to define the groups of variables. This can be a bit more cumbersome to do, as we specify them using nested dictionaries instead of plain text. However, it has the advantage that it can be processed in Python, which allows us to subset pandas objects using them. 

Specify groups:

In [23]:
g1 = GmsPy.Group('G_V01_NT_endo_always', v = [('pS', db['V01_NT_out']),
                                        ('pD', db['V01_NT_int']),
                                        ('qD', db['V01_NT_x'])])
g2 = GmsPy.Group('G_V01_NT_endo_base', v = [('qD', db['V01_NT_int']),
                                      ('qD', ('and', [db['V01_NT_inp'], ('not', db['V01_NT_x'])]))])
g3 = GmsPy.Group('G_V01_NT_exo_always', v = [('sigma', db['V01_NT_int']),
                                       ('sigma', db['V01_NT_out']),
                                       ('pD', db['V01_inp']),
                                       ('qS', db['V01_NT_out'])])
g4 = GmsPy.Group('G_V01_NT_exo_base', v = [('mu', db['V01_NT_map'])])
g5 = GmsPy.Group('G_V01_T_endo_always', v = [('lambda', db['V01_ES']),
                                       ('pD', db['V01_ES'])])
g6 = GmsPy.Group('G_V01_T_endo_base', v = [('qD', db['V01_T']),
                                     ('pD', db['V01_T'])])
g7 = GmsPy.Group('G_V01_T_exo_always', v = [('theta', db['V01_T']),
                                      ('mu', ('and', [db['V01_inp2T'], ('not', db['V01_dur'])])),
                                      ('sigma', db['V01_ES']),
                                      ('mu', db['V01_T2ESNorm'])])
g8 = GmsPy.Group('G_V01_T_exo_base', v = [('mu', ('and', [db['V01_T2ES'], ('not', db['V01_T2ESNorm'])])),
                                    ('mu', ('and', [db['V01_inp2T'], db['V01_dur']]))])
g9 = GmsPy.Group('G_V01_ACC_endo_base', v = [('qD', db['V01_inp'])])

Initialize settings with groups:

In [24]:
s = GmsPy.GmsSettings(**{'name': 'Test3', 'db': db, 'groups': {g.name: g for g in (g1,g2,g3,g4,g5,g6,g7,g8,g9)}})
s.Compile.run()
s.Precompiler.locals = {'techtype': "'logit'"}

Specify functions and blocks:

In [25]:
functions = read('V02_functions.gms') # read in file that declares functions
blocks = read('V02_blocks.gms') # read in equations for model.

Specify the endogenous/exogenous groups, specify blocks of equations:

In [26]:
s['g_endo'] = ['G_V01_NT_endo_always', 'G_V01_NT_endo_base', 'G_V01_T_endo_always', 'G_V01_T_endo_base', 'G_V01_ACC_endo_base']
s['g_exo'] = ['G_V01_NT_exo_always', 'G_V01_NT_exo_base', 'G_V01_T_exo_always', 'G_V01_T_exo_base']
s['blocks'] = ['M_V01_NT', 'M_V01_T_always', 'M_V01_T_base', 'M_V01_ACC']

We can now collect a standard selection of arguments:

In [27]:
s['args'] = s.stdArgs(blocks=blocks,functions=functions)
s['args'].keys()

dict_keys(['root', 'Test3_functions', 'Test3_declare', 'Test3_blocks', 'Test3_B_fix', 'Test3_B_unfix', 'Test3_B_model', 'Test3_B_solve'])

To solve the model, we simply use the ```self.write``` method again:

In [28]:
%%capture
s.write()

And set up model and solve:

In [29]:
m = GmsPy.GmsModel(ws=d['work'],**{'cns': 'CONOPT4'}) # use CONOPT4 to solve CNS models.
m.addlocal(db.name, db_str)
m.run(run = '\n'.join(s['text'].values()))